In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkFunctions").getOrCreate()

In [4]:
from pyspark import SparkFiles
url ="https://yelpproject4.s3.amazonaws.com/yelp_detail_review_merged3.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("yelp_detail_review_merged3.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------+-----+------------+--------------------+--------------------+--------------------+---------------+-------------------+-------------+
|         business_id|                name|             address|                city|               state|         postal_code|  latitude|   longitude|stars|review_count|          attributes|          categories|             user_id|   review_stars|               text|         date|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------+-----+------------+--------------------+--------------------+--------------------+---------------+-------------------+-------------+
|6iYb2HFDywm3zjuRg...| Oskar Blues Taproom|        921 Pearl St|             Boulder|                  CO|               80302|40.0175444|-105.28334

In [5]:
review_df = df.select(["text","stars"])
review_df.show()

+-------------------+-----+
|               text|stars|
+-------------------+-----+
| 'validated': False|  4.0|
|               null| null|
|               null| null|
|               null| null|
| 'validated': False|  4.0|
| 'validated': False|  4.0|
| 'validated': False|  4.0|
| 'validated': False|  4.0|
| 'validated': False|  4.0|
| 'validated': False|  4.0|
|               null| null|
| 'validated': False|  4.0|
|               null| null|
| 'validated': False|  4.0|
|               null| null|
|               null| null|
|               null| null|
| 'validated': False|  4.0|
| 'validated': False|  4.0|
| 'validated': False|  4.0|
+-------------------+-----+
only showing top 20 rows



In [17]:
from pyspark.sql.functions import regexp_extract, length
review_df = df.withColumn("text", regexp_extract("text", "\d+/\d+/\d+(?:\s)(.*)", 1))\
      .withColumnRenamed("stars", "label")\
      .select(["label", "date", "text"])
review_df = review_df.withColumn('review_length', length(review_df['text'])).dropna()
review_df.cache()
review_df.show()

+-----+-------------+----+-------------+
|label|         date|text|review_length|
+-----+-------------+----+-------------+
|  4.0| 'lot': False|    |            0|
|  4.0| 'lot': False|    |            0|
|  4.0| 'lot': False|    |            0|
|  4.0| 'lot': False|    |            0|
|  4.0| 'lot': False|    |            0|
|  4.0| 'lot': False|    |            0|
|  4.0| 'lot': False|    |            0|
|  4.0| 'lot': False|    |            0|
|  4.0| 'lot': False|    |            0|
|  4.0| 'lot': False|    |            0|
|  4.0| 'lot': False|    |            0|
|  4.0| 'lot': False|    |            0|
|  4.0| 'lot': False|    |            0|
|  4.0| 'lot': False|    |            0|
|  4.0| 'lot': False|    |            0|
|  4.0| 'lot': False|    |            0|
|  4.0| 'lot': False|    |            0|
|  4.0| 'lot': False|    |            0|
|  4.0| 'lot': False|    |            0|
|  4.0| 'lot': False|    |            0|
+-----+-------------+----+-------------+
only showing top

In [9]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'review_length'], outputCol='features')

In [10]:
# Create and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[tokenizer, stopremove, hashingTF, idf, clean_up])

In [11]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(review_df)
cleaned = cleaner.transform(review_df)

In [12]:
# Show label of ham spame and resulting features
cleaned.select(['label', 'features']).show()

AnalysisException: ignored

In [13]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

IllegalArgumentException: ignored

In [ ]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show(5)

In [ ]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)